# Get a Yelp API response

In [1]:
import requests
import pickle
import yaml
import pandas as pd

In [2]:
# To save the requests information
from mock_request.utils import save_requests_info

You must create the `./.private/api_key.yml` YAML file with the following structure:

```
- API_KEY:
    <YOUR API KEY VALUE>
```


In [3]:
# Read API key
with open("./.private/api_key.yml", "r") as f:
    API_KEY = yaml.load(f)[0]["API_KEY"]

In [4]:
# Define the base URL for the request
base_url = "https://api.yelp.com/v3/businesses/search"

# Set up the request headers -- API key is used here
headers = {"Authorization": "Bearer " + API_KEY}

In [5]:
# Define the request parameters
params = {
    "location": "Newark, NJ",
    "term": "laundromat",
    "limit": 5
}

In [6]:
response = requests.get(
    base_url,
    headers=headers,
    params=params
)

In [7]:
with open("./yelp-api-laundromat-newark-response.pkl", "wb") as f:
    pickle.dump(response, f)

# Is my key exposed in the response?


In [8]:
from inspect import ismethod

In [9]:
# Load pickled response
with open("./yelp-api-laundromat-newark-response.pkl", "rb") as f:
    loaded_response = pickle.load(f)
    

# List of methods to skip in the check
SKIP = [
    "__nonzero__", 
    "__setstate__", 
    "__repr__", 
    "__iter__", 
    "__module__", 
    "__init__", 
    "next"
]

# Check if API_KEY is in response attributes
for name in dir(loaded_response):
    
    if name in SKIP:
        continue
    
    # 
    att = getattr(loaded_response, name)
 
    if ismethod(att):
        att_text = str(att())
    elif not isinstance(att, str):
        att_text = str(att)
    else:
        att_text = att

    if API_KEY in att_text:
        print(name, ": FAIL.\tWARNING: KEY EXPOSED!")

    else:
        print(name, ": PASS.")

__attrs__ : PASS.
__bool__ : PASS.
__class__ : PASS.
__delattr__ : PASS.
__dict__ : PASS.
__dir__ : PASS.
__doc__ : PASS.
__enter__ : PASS.
__eq__ : PASS.
__exit__ : PASS.
__format__ : PASS.
__ge__ : PASS.
__getattribute__ : PASS.
__getstate__ : PASS.
__gt__ : PASS.
__hash__ : PASS.
__init_subclass__ : PASS.
__le__ : PASS.
__lt__ : PASS.
__ne__ : PASS.
__new__ : PASS.
__reduce__ : PASS.
__reduce_ex__ : PASS.
__setattr__ : PASS.
__sizeof__ : PASS.
__str__ : PASS.
__subclasshook__ : PASS.
__weakref__ : PASS.
_content : PASS.
_content_consumed : PASS.
apparent_encoding : PASS.
close : PASS.
content : PASS.
cookies : PASS.
elapsed : PASS.
encoding : PASS.
headers : PASS.
history : PASS.
is_permanent_redirect : PASS.
is_redirect : PASS.
iter_content : PASS.
iter_lines : PASS.
json : PASS.
links : PASS.
ok : PASS.
raise_for_status : PASS.
raw : PASS.
reason : PASS.
request : PASS.
status_code : PASS.
text : PASS.
url : PASS.


# Save objects for tests

### Correct requests/responses

In [10]:
test_params = params.copy()
test_params["base_url"] = base_url
test_params["pickle_path"] = "/home/adrian/Projects/mock-request/tests/files/yelp-api-laundromat-newark-response.pkl"
save_requests_info([test_params], "./requests_info_tests.json")

### Errored requests

In [11]:
# 
errored_requests = [
    # Wrong base URL
    {
        "pickle_file": "./yelp-api-wrong-base-url-response.pkl",
        "url": "https://api.yelp.com/v3/a_wrong_url",
        "headers": headers,
        "params": params
    },
    
    # Missing API key
    {
        "pickle_file": "./yelp-api-missing-key-response.pkl",
        "url": base_url,
        "params": params,
        "headers": {}
    },
    
    # Wrong location parameter name
    {
        "pickle_file": "./yelp-api-wrong-location-parameter-response.pkl",
        "url": base_url,
        "headers": headers,
        "params": {
            "locationnnnnnn": "Newark, NJ",
            "term": "laundromat",
            "limit": 5
        }
    }
]

error_codes = []
errored_response_paths = []

for er in errored_requests:
    
    path = er["pickle_file"]
    with open(path, "wb") as f:

        
        del(er["pickle_file"])
        
        # Pickle errored response
        r = requests.get(**er)
        pickle.dump(r, f)
        
        # Save error code and path
        error_codes.append(r.status_code)
        errored_response_paths.append(path)
        
# Create dataframe with error codes and paths
errors_tests = pd.DataFrame({
    "error_type": error_codes,
    "pickle_file": errored_response_paths
})

errors_tests.to_csv("./errors_tests.csv")